In [1]:
using Random


In [2]:
function show_board(sol::Array{Int})::Nothing
    n = length(sol)
    for i = 1:n
        for j = 1:n
            if sol[i] == j
                print("o ")
            else
                print(". ")
            end
        end
        println()
    end
end


show_board (generic function with 1 method)

In [3]:
function diagonals(sol::Array{Int})::Tuple{Array{Int}, Array{Int}}
    # ボード上の各対角線上のクイーンの数を決定する。

    n = length(sol)  # ボードの大きさ
    n_diag = 2 * n - 1  # 対角線の数

    diag_up = zeros(Int, n_diag)
    diag_dn = zeros(Int, n_diag)

    for i = 1:n
        # 行i列sol[j]のクイーンによって攻撃される斜め上対角線
        d = i + sol[i] - 1  # 対角線のインデックス
        diag_up[d] += 1

        # 行i列sol[j]のクイーンによって攻撃される斜め下対角線
        d = n + sol[i] - i  # 対角線のインデックス
        diag_dn[d] += 1
    end

    return diag_up, diag_dn
end


diagonals (generic function with 1 method)

In [4]:
function collisions(diag::Array{Int})::Int
    # 対角線上の衝突の総数を返す
    n_colls = 0
    for i in diag
        if i > 1
            # この対角線上に1つ以上のクイーンが既に配置されている
            n_colls += i - 1
        end
    end
    return n_colls
end


collisions (generic function with 1 method)

In [5]:
function exchange(i::Int, j::Int, sol::Array{Int}, diag_up::Array{Int}, diag_dn::Array{Int})::Nothing
    # 行iと行jのクイーンを交換し、それに伴う対角線情報を更新する
    n = length(sol)

    d = i + sol[i] - 1
    diag_up[d] -= 1
    d = j + sol[j] - 1
    diag_up[d] -= 1

    d = n + sol[i] - i
    diag_dn[d] -= 1
    d = n + sol[j] - i
    diag_dn[d] -= 1

    # iとjの位置を交換する
    sol[i], sol[j] = sol[j], sol[i]

    d = i + sol[i] - 1
    diag_up[d] += 1
    d = j + sol[j] - 1
    diag_up[d] += 1

    d = n + sol[i] - i
    diag_dn[d] += 1
    d = n + sol[j] - i
    diag_dn[d] += 1
end


exchange (generic function with 1 method)

In [6]:
function decrease(di::Int, dj::Int, ni::Int, nj::Int)::Int
    # クイーンが除去された際の、除去される衝突を計算する
    # di, dj: クイーンが現在置かれている対角線
    # ni, nj: 対角線上のクイーンの数
    delta = 0
    if ni >= 2
        delta -= 1
    end
    if nj >= 2
        delta -= 1
    end
    if di == dj && ni == 2
        delta += 1  # 1つ無駄に減らしてしまっている
    end
    return delta
end


decrease (generic function with 1 method)

In [7]:
function increase(di::Int, dj::Int, ni::Int, nj::Int)::Int
    # クイーンが置かれた際の、新しく発生する衝突を計算する
    # di, dj: クイーンが現在置かれている対角線
    # ni, nj: 対角線上のクイーンの数
    delta = 0
    if ni >= 1
        delta += 1
    end
    if nj >= 1
        delta += 1
    end
    if di == dj && ni == 0
        delta += 1  # 同じ対角線上にある
    end
    return delta
end


increase (generic function with 1 method)

In [8]:
function evaluate_move(i::Int, j::Int, sol::Array{Int}, diag_up::Array{Int}, diag_dn::Array{Int})::Int
    # 行iと行jのクイーンの交換を評価する

    delta = 0
    n = length(sol)

    # 移動が許されても、攻撃されない対角線
    upi = i + sol[i] - 1
    upj = j + sol[j] - 1
    delta += decrease(upi, upj, diag_up[upi], diag_up[upj])

    dni = n + sol[i] - i
    dnj = n + sol[j] - j
    delta += increase(dni, dnj, diag_dn[dni], diag_dn[dnj])

    # 攻撃されはじめる対角線
    upi = i + sol[j] - 1
    upj = j + sol[i] - 1
    delta += decrease(upi, upj, diag_up[upi], diag_up[upj])

    dni = n + sol[j] - i
    dnj = n + sol[i] - j
    delta += increase(dni, dnj, diag_dn[dni], diag_dn[dnj])

    return delta
end


evaluate_move (generic function with 1 method)

In [9]:
function find_move(n_iter::Int, tabu, best_colls, sol::Array{Int}, diag_up::Array{Int}, diag_dn::Array{Int}, n_colls::Int)
    # 最も良い移動(i, j)を返す。
    # 現在解からの可能な移動を全てチェックし、以下のようなものを選ぶ
    # ・TABUにない
    # ・TABUだが、aspiration criterionを満たす
    # 候補リストは、2行を入れ替える全ての可能性から成る。
    n = length(sol)
    best_delta = n  # 最も良い移動の値
    for i = 1:n-1
        for j = i + 1:n
            delta = evaluate_move(i, j, sol, diag_up, diag_dn)

            print("move ", i, "-", j, "-> delta=", delta, "; ")
            if tabu[i] >= n_iter
                print("move is tabu; ")
                if n_colls + delta < best_colls
                    print("aspiration criterion is satisfied; ")
                end
            end
            println()

            if tabu[i] < n_iter || n_colls + delta < best_colls  # move is not tabu or satisfies aspiration criteria
                if delta < best_delta
                    best_delta = delta
                    best_i = i
                    best_j = j
                end
            end
        end
    end
    return best_i, best_j, best_delta
end


find_move (generic function with 1 method)

In [10]:
function construct(sol::Array{Int})::Tuple{Array{Int}, Array{Int}}
    n = length(sol)
    n_diag = 2n - 1

    diag_up = zeros(Int, n_diag)
    diag_dn = zeros(Int, n_diag)

    cand = Array(1:n)
    trials = 10 * floor(log10(n))  # number of random trials
    for i = 1:n
        for t in trials
            col_id = rand(1:length(cand))
            col = cand[col_id]
            colls = diag_up[i + col - 1] + diag_dn[n - i + col]
            if colls == 0
                sol[i] = col
                diag_up[i + col - 1] += 1
                diag_dn[n - i + col] += 1
                deleteat!(cand, col_id)
                break
            else
                mincolls = 100000000000
                col_id = -1
                for j in 1:length(cand)
                    colls = diag_up[i + cand[j] - 1] + diag_dn[n - i + cand[j]]
                    if colls < mincolls
                        mincolls = colls
                        col = cand[j]
                        col_id = j
                    end
                end
                sol[i] = col
                diag_up[i + col - 1] += 1
                diag_dn[n - i + col] += 1
                deleteat!(cand, col_id)
            end
        end
    end
    return diag_up, diag_dn
end


construct (generic function with 1 method)

In [14]:
function main()
    LOG = true
    n = 10
    sol = Array(1:n)
    up, dn = construct(sol)
    n_colls = collisions(up) + collisions(dn)
    println("collisions of randomized greedy: ", n_colls)
    if LOG
        println("initial solution (random greedy) ")
        println("array: ", sol)
        show_board(sol)
        println("queens on upward diagonals: ", up)
        println("queens on downward diagonals: ", dn)
    end

    println("starting fast tabu search")
    # fast_tabu_search(sol, up, dn)
    # n_colls = collisions(up) + collisions(dn)
    # println("collisions: ", n_colls)
end
@showtime main()


collisions of randomized greedy: 3
initial solution (random greedy) 
array: [7, 1, 3, 5, 2, 4, 9, 6, 8, 10]
. . . . . . o . . . 
o . . . . . . . . . 
. . o . . . . . . . 
. . . . o . . . . . 
. o . . . . . . . . 
. . . o . . . . . . 
. . . . . . . . o . 
. . . . . o . . . . 
. . . . . . . o . . 
. . . . . . . . . o 
queens on upward diagonals: [0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1]
queens on downward diagonals: [0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 1, 1, 0, 0, 0, 1, 0, 0, 0]
starting fast tabu search
main(): 0.005192 seconds (1.70 k allocations: 49.812 KiB)
